# Récupérer des images depuis swisstopo

Le but de ce notebook est de tester la possibilité d'utiliser l'API STAC (SpatioTemporal Asset Catalog) afin de lister et télécharger des images d'intérêt depuis swisstopo.

In [11]:
import pystac
import requests

## Récupération du nom du jeu de données SWISSIMAGES via l'API

Il faut récupérer le nom du jeu de données [SWISSMAGES](https://www.swisstopo.admin.ch/fr/orthophotos-swissimage-10-cm). Les [infos générales](https://www.geo.admin.ch/fr/interface-rest-api-stac/) de l'API indiquent que l'adresse principale est `https://data.geo.admin.ch/api/stac/v0.9/` et la documentation détaillée est disponible [ici](https://data.geo.admin.ch/api/stac/static/spec/v0.9/api.html).

In [9]:
stac_api_url = ("https://data.geo.admin.ch/api/stac/v0.9")

On récupère les ids de toutes les collections disponibles ([doc](https://data.geo.admin.ch/api/stac/static/spec/v0.9/api.html#tag/Data/operation/getCollections)). Attention, les collections sont limités à 100 dans l'array `collections` retourné. Il faut utiliser le lien avec la `rel : "next"` contenu dans l'array `links` de l'objet contenant `{collections : [...], links : [...]}` !

In [33]:
# Stockage des ids
collection_ids = []

# GET sur les collections
response = requests.get(f"{stac_api_url}/collections")

# Vérification du succès
if response.status_code == 200:
    # Parse le json
    collections_data = response.json()

    # Extraire les ids des collections
    collection_ids.extend(collection["id"] for collection in collections_data.get("collections"))

    # Enchaîner avec les prochaines pages/liens
    while True:
        # Récupère le lien avec next ou retourne None si absent (dernier lien)
        next_link = next((link for link in collections_data.get("links") if link.get("rel") == "next"), None)
        if next_link:
            # Récupérer l'url
            next_url = next_link["href"]

            # Répéter les opérations précédentes
            response = requests.get(next_url)
            
            # Vérification du succès
            if response.status_code == 200:
                # Parse le json
                collections_data = response.json()

                # Extraire les ids des collections
                collection_ids.extend(collection["id"] for collection in collections_data.get("collections"))
            else:
                print("Échec. Erreur", response.status_code)
                break
        else:
            break  # Plus de lien "next"
else:
    print("Échec. Erreur", response.status_code)

print('\n'.join(collection_ids[0:3]))

['ch.agroscope.feuchtflaechenpotential-kulturlandschaft', 'ch.are.agglomerationsverkehr', 'ch.are.alpenkonvention']


On identifie les fournisseurs de données, qui viennent juste après le `ch.`. Ce sont généralement les abréviations allemandes des offices fédéraux (are (ODT), astra (OFROU), etc.).

In [39]:
fournisseurs = set(id.split(".")[1] for id in collection_ids)
print(sorted(fournisseurs))

['agroscope', 'are', 'armasuisse', 'astra', 'babs', 'bafu', 'bag', 'bak', 'bakom', 'baspo', 'bav', 'bazl', 'bfe', 'bfs', 'blw', 'ensi', 'meteoschweiz', 'pronatura', 'sem', 'swisstopo', 'swisstopo-vd', 'vbs']


Le founisseur qui nous intéresse est `swisstopo`, on regarde donc les jeux de données mis à disposition qui contiennent le mot `image`

In [43]:
for collection_id in collection_ids:
    if "swisstopo" in collection_id and "image" in collection_id:
        print(collection_id)

ch.swisstopo.swissimage-dop10


Le jeu de données des images est donc `ch.swisstopo.swissimage-dop10`

## Jeu de données SWISSIMAGE - Infos et téléchargement(s)

In [45]:
id_swimage = "ch.swisstopo.swissimage-dop10"

In [51]:
# GET sur le jeu de données
response = requests.get(f"{stac_api_url}/collections/{id_swimage}")
infos_jdd = response.json()